# Introduction
This documentation will go through building a new testing framework from the
ground up. This chapter will start with the problem statement, requirements
and provide motivations for writing our own framework. Writing a framework is
not easy and writing a framework that is adopted is nearly impossible. I hope
that providing the key motivations and drivers up front will help drive this
framework to success.

TOC
1. Intro
2. Building Blocks
3. Events
4. Executor
5. Text-Based Interface
6. Graphical Interface

## Problem Statement
We need automated tests. The tests will be written for both new and old 
code and ideally run on each commit. The tests should make it easy to 
determine the source of an bug (either with the system or the test). Unit 
tests are usually the preferred choice here for the following reasons:

1. Fast to execute
2. Isolated to a single unit

The rationale for \#1 is simple. The faster the tests the easier it is to run through
them all for each commit. \#2 I believe is a bit trickier to understand. First isolating
to a single unit is what makes the tests fast but it's more than that. Isolated to a 
single unit also means that when a test fails it almost certainly due to the unit under
tests and not due to some unrelated unit. So both these reasons certaintly
support our problem statement. But there are some disadvantages for unit tests.

1. It takes time to write test cases.
2. It's difficult to write tests for legacy code.
3. Tests require a lot of time for maintenance.
4. It can be challenging to test GUI code.
5. Unit testing can't catch all errors.

\#1 and \#3 are going to be a problem with any type of automated test strategy. \#4 is
not going to directly apply to our target system. That leaves us with \#2 and \#5. 
\#2 is going to work directly against our goal for writing tests for our current system.
The majority of the tests that will be written will be for legacy code. And I believe 
the style of our legacy code will make it even harder to write unit tests for them.
The legacy code is written in large units that will require large setup clauses that
ultimately will couple the tests to the code which will undoubtably lead to large
occurances of false positives. 

\#5 is a tradeoff of unit tests vs other tests. To get fast isolated tests you give up 
catching errors that come about when the system is integrated.

I am going to suggest that unit tests are not the right choice for most of our code base 
and instead suggest we do most testing at whitebox functional level. Im not sure if this 
is a real description for testing but essentially we will test the full system running but
allow us more freedom in how we invoke a test and determine the result. This is not to
say the framework shouldn't allow unit tests but it won't be the immediate focus.

!!! The real problem statement for the framework is here !!!
Most testing frameworks focus on synchronous 

To reimagine Requirement Verification Tests (RVTs) moving from
an imperative style to a declarative one. What do we mean by imperative vs declarative?
In their current form RVTs describe in extreme detail how to execute a test. 
They describe how to setup and how to configure the simulation, how to setup any 
preconditions, and when to restart a simulation. With all this detail it is usually
difficult to clearly see which lines are the true test and which lines are purely
setup. Additionally because starting a simulation is usually expensive tests are
organized to reuse a simulation which couples tests together in unspecified ways. This
means that when a test fails it can be hard to trace why. 

Additionally since each test procedure specifies the entire lifecyle of a test
it is hard to combine execution of tests. Authors sometimes will combine a few
tests into one to optimize test startup but it leaves a lot to be desired. 
Specifically a lot of these tests can be (or should be) run in parallel as that
usually how a system will ultimately be run in production. The current RVT format
provides no facilities for allowing for parallel test execution. 

A declarative approach on the other hand would specify the conditions necessary for the
test to be valid and let the testing apparatus to either figure out how to 
setup those conditions or to recognize when those conditions are valid and execute 
the test. 

This should provide atleast 3 immediate benefits:

1. Quicker test execution both in development and batches as the simulation in 
   most cases will not have to be restarted.
2. Tests that are not coupled
3. Clearer tests. 



## Requirement Verification Test (RVT) Background
This section will be a brief introduction of the current in use test framework called RVT. RVTs are the name given to the procedures executed by the Automation-V2 executor. RVT is a line-based, plaintext format with a keyword driven like syntax. 

RVTs are executed line by line using shell like syntax (i.e bash). This means the first space delimited token is the `command` and the rest are arguments to that `command`. In Automation-V2 the `command` is simply the name of a registered `BuildingBlock` which has a defined interface for (1) determining if it can accept the listed arguments and (2) a method to execute the `BuildingBlock`. When executed, a block will return either a `pass` or `fail` result. On `pass` the Automation-V2 executor will continue to the next line. On `fail` the framework will immediately stop the procedure execution. 

One last important concept of RVTs is there exists an `Observer` interface that allows listeners to register for execution events. An example of an event would be the `TestStart` and `TestCompletion` events. `Observers` allow for such things as TestReports to be created in a variety of formats. 

The following is a sample RVT procedure:

## Building Blocks
The BuildingBlock interface is the core concept of the Automation-V2 interface. It
is the main extension point used to extend the framework. It allows registering 
python code that will be executed when a line of RVT is encountered.

At it's core a BuildingBlock is very simple. It has 3 parts:

1. A function `name` for resolving the BuildingBlock name. Defaults to the classname
2. A function `check_syntax` to determine if the arguments are valid for this BuildingBlock
3. A function `execute` to run the block. Returns a `BlockResult` 


In [ ]:
# export

class BuildingBlock(object):
    ''' 
    The 'BuildingBlock' of the automation framework. Registers an function to
    be run during text execution.
    '''
    def name(self):
        '''Returns the name of the building block. The name is used
        as a first order lookup for the block'''
        return type(self).__name__
    
    def check_syntax(self, *args):
        '''Returns True if this BuildingBlock can support the arguments and False otherwise'''
        return True
    
    def execute(self, *args):
        '''Executes the block. Returns a BlockResult'''
        return BlockResult(False)
    
 

In [ ]:
#export

class BlockResult(object):
    '''
    The result of executing a BuildingBlock
    '''
    def __init__(self, passed, stdout="", stderr=""):
        self.passed = passed
        self.stdout = stdout
        self.stderr = stderr
        
    def __bool__(self):
        return self.passed
    
    def __str__(self):
        return "<BlockResult: %s, %s, %s>" % ('PASS' if self.passed else 'FAIL', self.stdout, self.stderr)
    

In [ ]:
# Unit tests
class TestBlock(BuildingBlock):
    pass

class TestBlock2(BuildingBlock):
    def name(self): return 'CustomName'
    
    
assert TestBlock().name() == 'TestBlock'
assert TestBlock2().name() == 'CustomName'

## Observers
Executing a test simply results in None if all tests have passed or a failed block result. 
We are going to want more detail as test executes and the `Observer` interface will give us that.
Observers will register themselves with an `ObserverManager` and get notified when key events
happen during test execution. One thing to note here is that we want observers to be open for extenson
and so we will use a convention that any method that begins with an 'on_' will be a observer event. Because
of this there is no specific Observer class as any python class with do.

This means an observer can implement any number of events.

In [ ]:
#export

from functools import partial
  
class Observer()
class ObserverManager(object):
    def __init__(self):
        self.observers = set()
    
    def add_observer(self, observer):
        self.observers.add(observer)
        
    def notify(self, event, *args, **kwargs):
        for observer in self.observers:
            if hasattr(observer, 'on_' + event):
                getattr(observer, 'on_' + event)(observer, *args, **kwargs)
            
    def __getattr__(self, name):
        if name.startswith('on_'):
            return partial(self.notify, name[3:])
        else:
            raise AttributeError
    
    # These are provided to document the usual events.
    # Observers are NOT limited to just these events
    
    def on_procedure_begin(self, *args, **kwargs):             self.notify('procedure_begin', *args, **kwargs)
    def on_step_start(self, *args, **kwargs):                  self.notify('step_start', *args, **kwargs)
    def on_step_end(self, *args, **kwargs):                    self.notify('step_end', *args, **kwargs)
    def on_procedure_end(self, *args, **kwargs):               self.notify('procedure_end', *args, **kwargs)
    def on_comment(self, *args, **kwargs):                     self.notify('comment', *args, **kwargs)
        

In [ ]:
# Unit Tests
manager = ObserverManager()

class TestObserver(object):
    def __init__(self):
        self.event_history = []
        
    def on_procedure_begin(self, *args, **kwargs):
        self.event_history.append(('procedure_begin', args[1:], kwargs))

    def on_custom_event(self, *args, **kwargs):
        self.event_history.append(('custom_event', args[1:], kwargs))
        
args1,   args2   = (1, 2, 3),        ('a', 'b', 'c')
kwargs1, kwargs2 = {'a': 1, 'b': 2}, {'1': 'a', '2': 'b'}

# Setup
observer = TestObserver()
manager.add_observer(observer)

# Execute
manager.on_procedure_begin(*args1, **kwargs1)
manager.on_comment()
manager.on_custom_event(*args2, **kwargs2)

# Assert
assert observer.event_history == [('procedure_begin', args1, kwargs1), ('custom_event', args2, kwargs2)]

Since we have two formats we will seperate the reading of the formats
from the evaluation of the test.

## Plaintext Reader
To read a plaintext format we simply can use the exising `shlex` module to
do most of the work. The only other noteable feature is that we support line
continuation. To continue a line simply end the like with `\` character.

In [ ]:
#export
import shlex

def read_plaintext(source):
    
    # handle line continuation
    source = source.replace('\\\n', '')

    return [shlex.split(line) 
            for line in source.splitlines()]


We now get an array of lines that are split by the rules of shell lexer. 

In [ ]:
read_plaintext(rvt_plaintext)

In [ ]:
#export

def is_empty(expr):
    return len(expr) == 0

def is_comment(expr):
    return len(expr) and expr[0].strip().startswith('#')

In [ ]:
assert is_empty([])
assert not is_empty(['ABC', '123'])

assert is_comment(['#', 'ABC'])
assert is_comment(['#ABC', '123'])
assert not is_comment(['ABC', '123'])

We need to define a few BuildingBlocks to test with.


In [ ]:
class EchoBlock(BuildingBlock):
    def execute(self, *args):
        stdout = f"{self.name()}, args: {args}"
        return BlockResult(True, stdout)
        
class StartSimulation(EchoBlock): pass
class StopSimulation(EchoBlock):  pass
class Verify(EchoBlock): pass
class SetValue(EchoBlock): pass
class SendCmd(EchoBlock): pass


Additionally we need a way to find a building block given an expression.

In [ ]:
#export

def all_subclasses(cls):
    return set(cls.__subclasses__()).union(
        [s for c in cls.__subclasses__() for s in all_subclasses(c)])

def find_block(expr):
    name = expr[0]
    args = expr[1:]
    blocks = [b() for b in all_subclasses(BuildingBlock)]
    matching = [b for b in blocks if b.name() == name and b.check_syntax(*args)]
    if matching:
        return matching[0]

In [ ]:
assert StartSimulation == type(find_block(['StartSimulation']))
assert Verify == type(find_block(['Verify', '1', '2']))
assert find_block(['Unknown']) is None

Lets write a helper function to execute a block

In [ ]:
def execute_block(expr):
    block = find_block(expr)
    if block:
        args = expr[1:]
        result = block.execute(args)
    else:
        result = BlockResult(False, stderr="Could not find block: " + " ".join([str(e) for e in expr])) 
    return result

In [ ]:
assert execute_block(['Verify', 1, 2, 3])
assert not execute_block(['Unknown', 1, 2, 3])

Now we can write the executor

In [ ]:
def execute_test(expressions):
    for expr in expressions:
        if is_empty(expr) or is_comment(expr):
            continue
            
        result = execute_block(expr)
        
        if not result:
            return result
            

Lets try and execute a test from before

In [ ]:

assert execute_test(read_plaintext(rvt_plaintext)) is None
assert execute_test(read_yaml(rvt_yaml)) is None

failing_rvt = '''
Verify Something
Unknown 123
'''

assert execute_test(read_plaintext(failing_rvt)).passed == False

Lets rewrite our test executor to take an observer

In [ ]:
def execute_test(expressions, observer):
    observer.on_procedure_begin(expressions)

    for expr in expressions:
        if is_empty(expr): continue
        if is_comment(expr):
            observer.on_comment(expr)
            continue
            
        observer.on_step_start(expr)
        try:
            result = execute_block(expr)
        except Exception as e:
            result = BlockResult(False, stderr=str(e))
        
        observer.on_step_end(result)
                
        if not result:
            observer.on_procedure_end(result)
            return result
        
    observer.on_procedure_end(True)
            

We can implement a simple print observer

In [ ]:
class StdoutObserver(Observer):
    def on_procedure_begin(self, procedure):
        print("Test Started")
        
    def on_step_start(self, step):
        print('RUN: ' + ' '.join([str(s) for s in step]), end='')
        
    def on_step_end(self, result):
        print(' [PASS]' if result else ' [FAIL]')
        
    def on_procedure_end(self, result):
        if isinstance(result, BlockResult):
            if len(result.stdout) > 0: print(result.stdout)
            if len(result.stderr) > 0: print('ERROR:', result.stderr, sep=' ')
        print('Test has ' + ('Passed' if result else 'Failed'))
        
    def on_comment(self, comment):
        ' '.join([str(s) for s in comment])

In [ ]:
observer_mgr = ObserverManager()
observer_mgr.add_observer(StdoutObserver)

# Lets run one and see if the test output is a little more interesting
execute_test(read_plaintext(rvt_plaintext), observer_mgr)

In [ ]:

# how about a failing test
execute_test(read_plaintext(failing_rvt), observer_mgr)